# install libraries

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

# Import helper libraries

In [3]:
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
%matplotlib inline

# connect to snowflake

In [12]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS_TRANSFORMED")
data

Existing connection object fetched: <snowflake.connector.connection.SnowflakeConnection object at 0x7f1877cfb730>
Please close the connection after use!
Reading dataframe from snowflake native connector


,ARRIVAL_DATE_TRANSFORMED,RESERVATION_STATUS_DATE_TRANSFORMED,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,2021-07-16,2020-10-17,City Hotel,1,272,2021,7,29,16,16-07-2021,Canceled,17-10-2020,2,0.272,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
1,2021-08-20,2020-10-17,City Hotel,1,307,2021,8,34,20,20-08-2021,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
2,2021-07-02,2020-10-17,City Hotel,1,258,2021,7,27,2,02-07-2021,Canceled,17-10-2020,2,0.258,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
3,2021-08-20,2020-10-17,City Hotel,1,307,2021,8,34,20,20-08-2021,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
4,2021-07-23,2020-10-17,City Hotel,1,279,2021,7,30,23,23-07-2021,Canceled,17-10-2020,2,0.279,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109706,2023-08-24,2023-09-10,Resort Hotel,0,269,2023,8,34,24,24-08-2023,Check.Out,10-09-2023,17,17.000,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
109707,2023-08-31,2023-09-10,Resort Hotel,0,212,2023,8,35,31,31-08-2023,Check.Out,10-09-2023,10,10.000,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
109708,2023-08-29,2023-09-12,Resort Hotel,0,204,2023,8,35,29,29-08-2023,Check.Out,12-09-2023,14,14.000,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
109709,2023-08-31,2023-09-14,Resort Hotel,0,161,2023,8,35,31,31-08-2023,Check.Out,14-09-2023,14,14.000,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06


In [13]:
df.columns = [x.lower() for x in df.columns]

In [14]:
df.columns

Index(['arrival_date_transformed', 'reservation_status_date_transformed',
       'hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'expected_arrival_date', 'reservation_status',
       'reservation_status_date', 'total_stay_nights', 'tally_days',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'total_guests', 'avg_rooms_per_night', 'total_room_nights',
       'meal', 'country', 'market_segment', 'distribution_channel',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'deposit_type',
       'days_in_waiting_list', 'customer_type', 'adr'],
      dtype='object')

# removing Canceletions and no-shows and keep City hotel data only


In [15]:
df = data[(data['is'] == 0) & (data['reservation_status'] !='No-Show')] 
df

KeyError: 'is_canceled'

In [7]:
df.groupby(['hotel','market_segment','reserved_room_type']).agg({'adr':'mean','reservation_status_date_transformed':'count'})


KeyError: 'hotel'